In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from PyLTSpice import SimCommander, RawRead
import pandas as pd
import glob
from scipy.fft import fft, fftfreq
from scipy.interpolate import interp1d
import os, shutil
from scipy.signal import windows
import time, re

In [ ]:
# ==========================================================
# CONFIG
# ==========================================================
asc_file = "coupled_wein_bridge.asc"
cmd = SimCommander(asc_file)
workdir = os.path.dirname(os.path.abspath(asc_file))
inter_R_values = np.linspace(100, 5000, 50)


# ==========================================================
# CLEAN UP OLD .raw FILES
# ==========================================================
import glob, os

old_raws = glob.glob(os.path.join(workdir, "coupled_wein_bridge_*.raw"))
if old_raws:
    print(f"🧹 Deleting {len(old_raws)} old .raw file(s)...")
    for f in old_raws:
        try:
            os.remove(f)
        except Exception as e:
            print(f"⚠️ Could not delete {f}: {e}")
else:
    print("No old .raw files found — clean start.")

time.sleep(10)

# ==========================================================
# PHASE 1 — RUN SIMULATIONS
# ==========================================================
print("\n=== Running all LTspice simulations ===")
for i, R in enumerate(inter_R_values, 1):
    print(f"▶️ [{i}/{len(inter_R_values)}] Running inter_R = {R:.2f} Ω")
    cmd.set_parameters(inter_R=R)
    task = cmd.run()
    time.sleep(0.5)  # short pause between launches

print("\n✅ All simulations launched! Waiting a bit for LTspice to finish writing files...")
time.sleep(10)  # optional buffer to ensure all files finish writing



# ==========================================================
# PHASE 2 — BUILD RAW FILE MAPPING
# ==========================================================
all_raw_files = glob.glob("coupled_wein_bridge_*.raw")
raw_files = [f for f in all_raw_files if ".op" not in f] 
raw_files =  sorted(raw_files, key=lambda x: int(re.search(r'(\d+)', x).group()))
print(f"\nDetected {len(raw_files)} .raw files in total.")

# LTspice usually names them 1.raw, 2.raw, ..., matching order of execution.
# So we can assume ascending order corresponds to increasing R.
raw_mapping = list(zip(inter_R_values[:len(raw_files)], raw_files))

In [ ]:
 """"
raw_mapping = []

for R in inter_R_values:
    print(f"Running with inter_R = {R:.2f} Ω")
    cmd.set_parameters(inter_R=R)
    cmd.run()
    time.sleep(2)

    all_raw_files = sorted(glob.glob("coupled_wein_bridge_*.raw"), key=os.path.getmtime)
    if not all_raw_files:
        print("⚠️ No raw file found!")
        continue

    latest_raw = all_raw_files[-1]
    new_raw_name = f"coupled_wein_bridge_R{R:.2f}.raw"
    shutil.copy(latest_raw, new_raw_name)
    print(f"✅ Saved {new_raw_name}")

    raw_mapping.append((R, new_raw_name))
""""""

In [ ]:
def compute_order_parameter(raw, node_list):
    t = raw.get_trace("time").get_wave()
    voltages = []
    for node in node_list:
        try:
            v = raw.get_trace(node).get_wave()
            voltages.append(v)
        except KeyError:
            print(f"Warning: node {node} not found in {raw.filename}")
    if len(voltages) == 0:
        return np.nan
    voltages = np.array(voltages)

    # Compute analytic signal and instantaneous phase
    analytic = hilbert(voltages, axis=1)
    phases = np.unwrap(np.angle(analytic), axis=1)

    # Kuramoto order parameter R(t)
    R_t = np.abs(np.mean(np.exp(1j * phases), axis=0))
    # Average R(t) over last 50% of samples (steady state)
    steady_idx = int(0.0 * len(R_t))
    R_mean = np.mean(R_t[steady_idx:])
    return R_mean

In [ ]:
# Correct node names from your simulation
layer1_nodes = ["V(l1vout1)", "V(l1vout2)", "V(l1vout3)", "V(l1vout4)", "V(l1vout5)"]
layer2_nodes = ["V(l2vout1)", "V(l2vout2)", "V(l2vout3)", "V(l2vout4)", "V(l2vout5)"]

In [ ]:
results = []
for R, rawfile in raw_mapping:
    print(f"Analyzing {os.path.basename(rawfile)} (R={R:.2f})")
    raw = RawRead(rawfile)
    R1 = compute_order_parameter(raw, layer1_nodes)
    R2 = compute_order_parameter(raw, layer2_nodes)
    results.append((R, R1, R2))

df = pd.DataFrame(results, columns=["Resistor", "R1", "R2"]).dropna().sort_values("Resistor")

In [ ]:
# ================================================================
#  PLOT RESULTS
# ================================================================
plt.figure(figsize=(7, 5))
plt.plot(df["Resistor"], df["R1"], "o--", label="Layer 1 (Environmental)")
plt.plot(df["Resistor"], df["R2"], "s-", label="Layer 2 (Dynamical)")
plt.xlabel("Coupling Resistor (Ω)")
plt.ylabel("Order Parameter R")
plt.title("Double-Layer Wien Bridge Network Synchronization")
plt.grid(True)
plt.xscale("log")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# ================================================================
#  PLOT RESULTS
# ================================================================
plt.figure(figsize=(7, 5))
plt.plot(1/df["Resistor"], df["R1"], "o--", label="Layer 1 (Environmental)")
plt.plot(1/df["Resistor"], df["R2"], "s-", label="Layer 2 (Dynamical)")
plt.xlabel("Coupling Resistor (k)")
plt.ylabel("Order Parameter R")
plt.title("Double-Layer Wien Bridge Network Synchronization")
plt.xscale("log")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()